In [1]:
from pyspark import SparkContext, SparkConf # Spark
from pyspark.sql import SparkSession # Spark SQL
from pyspark.sql.types import *
from pyspark.sql.functions import *

sc = SparkContext.getOrCreate()

if (sc is None):
    sc = SparkContext(master="local[3]", appName="Introduction to Apache Spark")
spark = SparkSession(sparkContext=sc)

#Nodes TABLE
scNodes = StructType([
StructField("node_id",IntegerType()),
StructField("x",FloatType()),
StructField("y",FloatType())
])

#Nodes DATA
dataNodes = sc.textFile('nodes.csv')
dataNodes = dataNodes.map(lambda x: x.split(','))
dataNodes = dataNodes.map(lambda x: [int(x[0]),float(x[1]), float(x[2])])
dfNodes = spark.createDataFrame(dataNodes,schema=scNodes) 
dfNodes.createOrReplaceTempView("dataNodes")

sc2 = SparkContext.getOrCreate()

if (sc2 is None):
    sc2 = SparkContext(master="local[3]", appName="Introduction to Apache Spark")
spark2 = SparkSession(sparkContext=sc2)

#Edges TABLE
scEdges = StructType([
StructField("f",IntegerType()),
StructField("t",IntegerType()),
StructField("p",FloatType())
])

#Edges DATA
dataEdges = sc2.textFile('edges.csv')
dataEdges = dataEdges.map(lambda x: x.split(','))
dataEdges = dataEdges.map(lambda x: [int(x[0]),int(x[1]), float(x[2])])
dfEdges = spark2.createDataFrame(dataEdges,schema=scEdges) 
dfEdges.createOrReplaceTempView("dataEdges")

sc3 = SparkContext.getOrCreate()

if (sc3 is None):
    sc3 = SparkContext(master="local[3]", appName="Introduction to Apache Spark")
spark3 = SparkSession(sparkContext=sc3)

#Path TABLE
scPath = StructType([
StructField("path",StringType()),
StructField("p",StringType())
])

#Path DATA
dataPath = sc3.textFile('path.csv')
dataPath = dataPath.map(lambda x: x.split(','))
dataPath = dataPath.map(lambda x: [x[0],x[1]])
dfPath = spark3.createDataFrame(dataPath,schema=scPath) 
dfPath.createOrReplaceTempView("dataPath")

In [2]:
from time import sleep
from json import dumps
from kafka import KafkaProducer
import random

producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                         value_serializer=lambda x:
                         dumps(x).encode('utf-8'))
idx=1
key = "ok"
for e in range(dfNodes.count()):
    result = dfNodes.select("node_id","x","y").filter("node_id=" + str(idx))
    txt = "no" + "," + str(result.first().node_id) + "," + str(result.first().x) + "," + str(result.first().y)
    data = txt
    print(data)
    producer.send(key, value=data)
    idx=idx+1

from pyspark.sql.window import Window  
w = Window.orderBy("f") 
df_index = dfEdges.withColumn("id", row_number().over(w))
###
idx=1
for e in range(df_index.count()):
    result = df_index.select("f","t","p").filter("id=" + str(idx))
    txt = "node" + "," + str(result.first().f) + "," + str(result.first().t) + "," + str(result.first().p)
    data = txt
    print(data)
    producer.send(key, value=data)
    idx=idx+1
idx=1
w = Window.orderBy("p") 
dfPath = dfPath.withColumn("id", row_number().over(w))
for e in range(dfPath.count()):
    result = dfPath.select("path").filter("id=" + str(idx))
    txt = "path" + "," + str(result.first().path) + "," + "no" + "," + "no"
    data = txt
    print(data)
    producer.send(key, value=data)
    idx=idx+1

no,1,5.0,2.0
no,2,5.0,4.0
no,3,7.0,5.0
no,4,8.0,3.0
no,5,9.0,1.0
no,6,11.0,4.0
no,7,13.0,1.0
no,8,9.0,6.0
no,9,13.0,5.0
no,10,6.0,0.0
no,11,14.0,3.0799999237060547
no,12,14.890000343322754,6.21999979019165
no,13,5.78000020980835,6.489999771118164
no,14,8.859999656677246,3.2899999618530273
no,15,6.909999847412109,1.399999976158142
no,16,3.240000009536743,4.639999866485596
no,17,9.5,-0.949999988079071
no,18,10.9399995803833,1.7400000095367432
no,19,2.9100000858306885,0.41999998688697815
no,20,14.210000038146973,0.0
node,1,2,2.0
node,1,4,3.1600000858306885
node,1,10,2.240000009536743
node,1,19,2.619999885559082
node,2,3,2.240000009536743
node,2,13,2.609999895095825
node,2,16,1.8799999952316284
node,3,4,2.240000009536743
node,3,6,4.119999885559082
node,3,8,2.240000009536743
node,3,14,2.5299999713897705
node,4,5,2.240000009536743
node,5,6,3.609999895095825
node,5,14,2.299999952316284
node,5,15,2.130000114440918
node,6,7,3.609999895095825
node,6,9,2.240000009536743
node,6,11,3.14000010490417